7-параметрическая задача по определению концентрации ионов $Cu^{2+}$', '$Ni^{2+}$', '$Pb^{2+}$', '$Co^{2+}$', '$Al^{3+}$', '$Cr^{3+}$', '$NO_3^-$ по спектрам ФЛ УТ.

In [33]:
import torch
import numpy as np
import torch.nn as nn
import pandas as pd
import re
import os
#import tensorflow as tf
import sklearn
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import median_absolute_error
from sklearn.metrics import r2_score
import torchvision.transforms as transforms
from torchvision.io import read_image
from torch.utils.data import Dataset
from sklearn.model_selection import KFold
import random
import torch.optim as optim
import torch.nn.functional as F
import matplotlib.pyplot as plt
import os
from torch.utils.data import DataLoader
from itertools import chain
import pandas as pd 
import glob
import matplotlib.pyplot as plt
from tqdm import trange
import datetime
from torcheval.metrics import R2Score


# Dataset

In [22]:
class CDs_2D_dataset(Dataset):
    def __init__(self, annotations_file, spec_dir, transform=None, target_transform=None):
      '''annotations_file - the file which contains lables of the samples included in training/validation/test sets'''
      self.spec_labels = pd.read_csv(annotations_file, sep=',', decimal=",").iloc[:,1:] # labels (concentrations for 4 ions) for all the samples from annotation file (Y_ions.csv)
      self.spec_number = pd.read_csv(annotations_file, sep=',', decimal=",").iloc[:,0] # numbers for all the samples from annotation file (Y_ions.csv)
      self.spec_dir = spec_dir # folder where csv files are located
      self.transform = transform 
      self.target_transform = target_transform

    def __len__(self):
        return len(self.spec_labels)#length of the dataset

    def __getitem__(self, idx):
        label = self.spec_labels.iloc[idx] # get the label of the sample via sample's index
        
        sp = np.array(pd.read_csv(self.spec_dir + str(self.spec_number[idx])+'.csv').T.iloc[1:,:], dtype='float32') # get the EEM of the sample via sample's index
        #sp = np.array(pd.read_csv(self.spec_dir + str(self.spec_number[idx])+'_CorrectionData'+'.csv', skiprows=38, sep=';', decimal=",").T.iloc[1:-1,:], dtype='float32') # get the EEM of the sample via sample's index

        sp[sp<0]=0 # here we zero negative values of intensities
        spec = torch.from_numpy(sp).unsqueeze(0) # add dimension for channels of cnn
        
        if self.transform:
            spec = self.transform(spec)
        if self.target_transform:
            label = self.target_transform(label)
            
        return spec, torch.from_numpy(np.array(label, dtype='float32')) # return spectrum and corresponding labels
     

In [23]:
# file_x = '/Users/galinacugreeva/Desktop/УТ 7 параметров/CD_HM_dataset/'#1000_CorrectionData.csv'
# file_y = '/Users/galinacugreeva/Desktop/УТ 7 параметров/CD_HM_dataset/Y_ions.csv'

file_x = '../CD_HM_dataset/'
file_y = '../CD_HM_dataset/Y_ions.csv'

training_data =  CDs_2D_dataset(file_y, file_x)

train_dataloader = DataLoader(training_data, batch_size=1, shuffle=True)

# смотреть на размерности
for i, j in train_dataloader:
    print(i.shape[0], i.shape[1], j.shape[0], j.shape[1])
    print(i[0]), print(i[0].shape)
    print(j)
    break
    
  

1 1 1 7
tensor([[[ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
         [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
         [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
         ...,
         [ 0.0000, 20.4750,  0.0000,  ...,  0.2660,  0.0000,  0.0000],
         [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.3290,  0.0000],
         [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000]]])
torch.Size([1, 201, 27])
tensor([[ 3.0000,  3.0000,  4.5000,  6.0000,  6.0000,  1.5000, 55.5000]])


## Calculate mean and std

In [24]:
def mean_std(train_dataloader):
    mean = 0.0
    for specs, _ in train_dataloader:
        batch_samples = specs.size(0) 
        specs = specs.view(batch_samples, specs.size(1), -1)
        mean += specs.mean(2).sum(0)
    mean = mean / len(train_dataloader.dataset)

    var = 0.0
    for specs, _ in train_dataloader:
        batch_samples = specs.size(0)
        specs = specs.view(batch_samples, specs.size(1), -1)
        var += ((specs - mean.unsqueeze(1))**2).sum([0,2])
    #std = torch.sqrt(var / (len(train_dataloader.dataset)*500*41))
    std = torch.sqrt(var / (len(train_dataloader.dataset)*200*27))

    return mean, std

# Model

## Useful functions

In [25]:
def reset_weights(m):
    '''
    Try resetting model weights to avoid weight leakage.
    '''
    for layer in m.children():
        if hasattr(layer, 'reset_parameters'):
            print(f'Reset trainable parameters of layer = {layer}')
            layer.reset_parameters()

In [26]:
def write_predictions(N, model_name, split_path, optimizer, dataloader, dset):
    
    #load the model
    checkpoint = torch.load(split_path + 'model'+model_name+'.pth')
    N.load_state_dict(checkpoint['model_state_dict'])
    
    #load optimizer
    optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
    
    last_best_epoch = checkpoint['epoch'] #the epoch with minimum loss function that occured during training
    loss = checkpoint['loss']
    N.eval()

    y_ae = np.zeros((1,7))
    y_ae_true = np.zeros((1,7))
    
    #write the outputs of the model
    for specs, labels in dataloader:
        outputs = N(specs) # get the outputs from the network
        outputs[outputs<0]=0 #we zero negative outputs as they are impossible for concentration values
        ae = outputs.detach().numpy()
        ae_true = labels.detach().numpy()
        #np.concatenate((y_ae, ae), axis=0)
        y_ae = np.concatenate((y_ae, ae), axis=0) # columns with networks's output for the dataloader
        y_ae_true = np.concatenate((y_ae_true, ae_true), axis=0) # columns with true output for the dataloader

    a = ['Cu','Ni', 'Pb', 'Al', 'Co', 'Cr','NO3']
    #a = ['pH']
    pd.DataFrame(y_ae).to_csv(split_path + 'Y_out_'+'_'+dset+'.csv',sep=',', header = a)
    pd.DataFrame(y_ae_true).to_csv(split_path + 'Y_true_'+'_'+dset+'.csv',sep=',', header = a)

In [27]:
import math

def calculate_metrics(model, loader):
    
    for specs, targets in loader:
        preds = model(specs)
        preds[preds<0]=0
    
        loss_mae = nn.L1Loss()
        mae = loss_mae(preds, targets)
    
        loss_rmse = nn.MSELoss()
        rmse = loss_rmse(preds, targets)**0.5
    
        r2 = R2Score()
        r2.update(preds, targets)
        r2 = r2.compute()
    
    return mae, rmse, r2

# 1. Cross-validation

In [28]:
gen_path = 'C:/Users/Liquid_Lab/Desktop/7-параметрическая задача УТ/' 

case_name = '6_2DCNN_str5_32neur_leakyrelu' + '_' + str(datetime.date.today())
#for cat in ['Cu', 'Cr', 'Ni', 'anions']:
cnn_2d_path = case_name+'/'#+cat+'/'

Y = pd.read_csv(gen_path+'CD_HM_dataset/'+'Y_ions'+'.csv', sep=',')


k_folds = [[42,12],[612,45],[72,172]] 

# for fold in k_folds:
#     split_path = gen_path+cnn_2d_path+'split_'+ str(fold[0])+'_'+str(fold[1])+ '/'
#     os.makedirs(split_path, exist_ok=True)

#     Y_trn, Y_30 = train_test_split(Y, test_size=0.3, random_state=fold[0])
#     Y_vld, Y_tst = train_test_split(Y_30, test_size = 0.3333, random_state=fold[1])

#     a = ['sample_number','Cu','Ni', 'Pb', 'Al', 'Co', 'Cr','NO3']

#     pd.DataFrame(Y_trn).to_csv(split_path + 'Y_trn'+'.csv',sep=',', index=False, header = a)
#     pd.DataFrame(Y_vld).to_csv(split_path + 'Y_vld'+'.csv',sep=',', index=False, header = a)
#     pd.DataFrame(Y_tst).to_csv(split_path + 'Y_tst'+'.csv',sep=',', index=False, header = a)



# wandb

In [29]:
import wandb

wandb.login()


wandb: WARNING Calling wandb.login() after wandb.init() has no effect.


True

# Training

In [30]:
def train(model, lr, l2_lambda, k_folds, case_name, mnozh_init, epochs_num):

    print('Start in', str(datetime.datetime.now()))

    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    print('Using device:', device)
        
    loss_function = torch.nn.MSELoss().cuda()
    optimizer = optim.Adam(model.parameters(), lr=lr)
    
    
    for fold in k_folds:
        
        split_path = gen_path+cnn_2d_path+'split_'+ str(fold[0])+'_'+str(fold[1])+ '/'

        training_data = CDs_2D_dataset(split_path+'Y_trn'+'.csv',gen_path + 'CD_HM_dataset/')
        train_dataloader = DataLoader(training_data, batch_size=256, shuffle=True)
        mean, std = mean_std(train_dataloader)
    
        #Data import and normalization Basic
    
        training_data = CDs_2D_dataset(split_path+'Y_trn'+'.csv', gen_path + 'CD_HM_dataset/', transform= transforms.Compose([transforms.Normalize(mean, std)]))
    
        validation_data = CDs_2D_dataset(split_path+'Y_vld'+'.csv', gen_path + 'CD_HM_dataset/', transform= transforms.Compose([transforms.Normalize(mean, std)]))
        test_data = CDs_2D_dataset(split_path+'Y_tst'+'.csv', gen_path + 'CD_HM_dataset/', transform= transforms.Compose([transforms.Normalize(mean, std)]))
    
        train_dataloader = DataLoader(training_data, batch_size=64, shuffle=True)
        validation_dataloader = DataLoader(validation_data, batch_size=64, shuffle=True)
        test_dataloader = DataLoader(test_data, batch_size=64, shuffle=True)

        print(mean, std)

    
        for init_number in range(0, mnozh_init): #Это я делаю множественную инициализацию весов сети
    
            init_path = split_path + str(init_number)+'/'
            os.makedirs(init_path, exist_ok=True)
    
            test_stop = 100 #stopping criterion
            max_val_loss = 10000.0

            wandb.init(project = case_name)
        
            split_name = 'split_'+ str(fold[0])+'_'+str(fold[1])
            init_name = '_' + str(init_number)
            model_name = '_2D_CNN'

            wandb.run.name = split_name + init_name + model_name+'_reg_'+str(l2_lambda)
            wandb.run.save()
    
            for epoch_step in range(0, epochs_num, test_stop):
                
                if epoch_step!=0:
                    checkpoint = torch.load(init_path + 'model'+model_name+'.pth')
                    model.load_state_dict(checkpoint['model_state_dict'])
                    optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
                    last_best_epoch = checkpoint['epoch']
                    loss = checkpoint['loss']
                    model.train()
                    
                    if last_best_epoch + test_stop > ep:
                        for ep in range(epoch_step, epoch_step+test_stop):
                            for _, data in enumerate(train_dataloader, 0): # get bacth
                                inputs, labels = data # parse batch
                                optimizer.zero_grad() # sets the gradients of all optimized tensors to zero.
                                outputs = model(inputs) # get outputs

                                loss = loss_function(outputs, labels) # calculate loss
                                l2_norm = sum(p.pow(2.0).sum() for p in model.parameters())
                                loss = loss + l2_lambda * l2_norm
                                loss.backward() # calculate gradients
                            
                                optimizer.step() # performs a single optimization step (parameter update).
    
                            dl = 0
                            val_loss = 0.0
                            for specs, labels in validation_dataloader: 
                                val_loss += loss_function(model(specs),labels)
                                dl+=1
                            val_loss = val_loss/dl
                            
                            if val_loss.item() <= max_val_loss:
                                torch.save({'epoch': ep,
                                  'model_state_dict': model.state_dict(),
                                  'optimizer_state_dict': optimizer.state_dict(),
                                  'loss': loss}, init_path + 'model'+model_name+'.pth')
                                max_val_loss = val_loss.item()
                            wandb.log({"trn_loss": loss, "vld_loss": val_loss})

                    else: continue    
                print(epoch_step)
                if epoch_step==0:
                    model.apply(reset_weights)
    
                    for ep in range(epoch_step, test_stop):
                        for _, data in enumerate(train_dataloader, 0): # get bacth
                            inputs, labels = data # parse batch
                            optimizer.zero_grad() # sets the gradients of all optimized tensors to zero.
                            outputs = model(inputs) # get outputs
                            loss = loss_function(outputs, labels) # calculate loss
                            loss.backward() # calculate gradients
                            optimizer.step() # performs a single optimization step (parameter update).
    
                        dl = 0
                        val_loss = 0.0
                        for specs, labels in validation_dataloader:
                            val_loss += loss_function(model(specs),labels)
                            dl+=1
                        val_loss = val_loss/dl
    
                        if val_loss.item() <= max_val_loss:
                            torch.save({'epoch': ep,
                              'model_state_dict': model.state_dict(),
                              'optimizer_state_dict': optimizer.state_dict(),
                              'loss': loss}, init_path + 'model'+model_name+'.pth')
                            max_val_loss = val_loss.item()
                        wandb.log({"trn_loss": loss, "vld_loss": val_loss})
    
            write_predictions(model, model_name, init_path, optimizer, train_dataloader, dset = 'trn')
            write_predictions(model, model_name, init_path, optimizer, validation_dataloader, dset ='vld')
            write_predictions(model, model_name, init_path, optimizer, test_dataloader, dset ='tst')

            trn_metrics = calculate_metrics(model, train_dataloader)
            vld_metrics = calculate_metrics(model, validation_dataloader)
            tst_metrics = calculate_metrics(model, test_dataloader)

            wandb.log({"trn_mae": float(trn_metrics[0]), "trn_rmse": float(trn_metrics[1]), "trn_r2": float(trn_metrics[2])})
            wandb.log({"vld_mae": float(vld_metrics[0]), "vld_rmse": float(vld_metrics[1]), "vld_r2": float(vld_metrics[2])})
            wandb.log({"tst_mae": float(tst_metrics[0]), "tst_rmse": float(tst_metrics[1]), "tst_r2": float(tst_metrics[2])})
            
            wandb.log({"epoch": ep})
    
            wandb.finish()
            
            print(epoch_step, fold)
            print(str(datetime.datetime.now()))
            
    print('End in', str(datetime.datetime.now()))



# 2. Model

In [31]:
class twoD_CNN_new_leakyrelu_32(nn.Module):
    
    def __init__(self):
        
        super().__init__()

        self.CNN = nn.Sequential(
            nn.Conv2d(in_channels=1, out_channels=3, kernel_size=5, stride=5),
            nn.LeakyReLU(),
            nn.MaxPool2d(2), 
            nn.Flatten(),
            nn.Linear(3 * 20 * 2, 32), 
            nn.LeakyReLU(), 
            nn.Linear(32, 7) 

        )

    def forward(self, x):
        x = self.CNN(x)
        return x 

In [34]:
k_folds = [[612,45],[72,172]] 

train(model=twoD_CNN_new_leakyrelu_32(), 
      lr=0.001, 
      l2_lambda = 0.001,
      k_folds=k_folds,
      case_name=case_name,
      mnozh_init=1, 
      epochs_num=5000)

Start in 2024-09-06 14:04:33.157493
Using device: cuda
tensor([34.0707]) tensor([119.0244])


trn_loss,█▄▆▄▅▄▃▃▃▃▃▂▃▃▂▂▃▃▄▃▂▃▁▃▃▃▂▁▁▃▂▂▄▂▇▃▂▃▁▃
vld_loss,█▆▄▄▃▃▃▂▂▂▂▁▁▁▂▁▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
trn_loss,2.83233
vld_loss,3.5609


0
Reset trainable parameters of layer = Conv2d(1, 3, kernel_size=(5, 5), stride=(5, 5))
Reset trainable parameters of layer = Linear(in_features=120, out_features=32, bias=True)
Reset trainable parameters of layer = Linear(in_features=32, out_features=7, bias=True)
100
200


wandb: Network error resolved after 0:01:54.347833, resuming normal operation.
wandb: Network error resolved after 0:00:41.435756, resuming normal operation.


300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
1900
2000
2100
2200


epoch,▁
trn_loss,██▆▇▅▄▃▄▄▄▅▄▃▃▂▃▃▃▁▃▄▂▅▂▆▄▂▁▂▃▃▃▃▂▃▂▂▃▄▄
trn_mae,▁
trn_r2,▁
trn_rmse,▁
tst_mae,▁
tst_r2,▁
tst_rmse,▁
vld_loss,█▆▅▄▃▃▂▂▂▂▂▂▂▂▂▂▂▂▂▂▁▁▁▁▂▁▁▁▁▁▂▁▁▁▁▁▂▁▁▁
vld_mae,▁
vld_r2,▁


4900 [612, 45]
2024-09-07 04:27:23.406424
tensor([34.3572]) tensor([111.3454])


0
Reset trainable parameters of layer = Conv2d(1, 3, kernel_size=(5, 5), stride=(5, 5))
Reset trainable parameters of layer = Linear(in_features=120, out_features=32, bias=True)
Reset trainable parameters of layer = Linear(in_features=32, out_features=7, bias=True)
100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600


epoch,▁
trn_loss,█▇▄▇▂▁▃▂▃▃▁▃▃▃▄▁▂▃▁▂▁▄▂▁▄▁▂▁▃▂▂▂▃▂▃▁▂▆▂▂
trn_mae,▁
trn_r2,▁
trn_rmse,▁
tst_mae,▁
tst_r2,▁
tst_rmse,▁
vld_loss,█▆▄▃▃▂▂▂▂▂▂▂▁▁▂▁▁▁▁▁▂▂▁▁▁▁▂▁▁▁▁▁▁▁▁▁▁▁▁▂
vld_mae,▁
vld_r2,▁


4900 [72, 172]
2024-09-07 15:01:53.968557
End in 2024-09-07 15:01:53.968557
